In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential


from google.colab import drive
import numpy as np
import pandas as pd

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [6]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
file_path = "/content/gdrive/My Drive/shakespeare.txt"

with open(file_path,"r") as f:
  text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [8]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [10]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [11]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [12]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [13]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [14]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [16]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [18]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [19]:
char_rnn.fit(X,Y, epochs=20, batch_size=1024)

Epoch 1/20
38/38 [==============================] - 1s 17ms/step - loss: 3.2234
Epoch 2/20
38/38 [==============================] - 1s 15ms/step - loss: 2.6134
Epoch 3/20
38/38 [==============================] - 1s 15ms/step - loss: 2.2121
Epoch 4/20
38/38 [==============================] - 1s 15ms/step - loss: 2.0026
Epoch 5/20
38/38 [==============================] - 1s 15ms/step - loss: 1.8752
Epoch 6/20
38/38 [==============================] - 1s 15ms/step - loss: 1.7731
Epoch 7/20
38/38 [==============================] - 1s 15ms/step - loss: 1.6884
Epoch 8/20
38/38 [==============================] - 1s 15ms/step - loss: 1.6146
Epoch 9/20
38/38 [==============================] - 1s 15ms/step - loss: 1.5538
Epoch 10/20
38/38 [==============================] - 1s 15ms/step - loss: 1.4982
Epoch 11/20
38/38 [==============================] - 1s 15ms/step - loss: 1.4567
Epoch 12/20
38/38 [==============================] - 1s 15ms/step - loss: 1.4010
Epoch 13/20
38/38 [==================

Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [20]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
      probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [21]:
sim = sample_text(char_rnn,sample_length=500) 

In [22]:
print(sim)

 you dress'd yourself? Hath it slept since
    I have hings, herevent, showl us his.
  FIRSS WITCH. What, it the be whele thou pcotled noblan,
    That hann ammiscoutibe a ware of kent
    The the word, is you, plocain'd mintere,
    Seceoubine those Burdid. Reven they relick,
    Which sould brain ey held.                                                                             Enter there
    They uth all vanares I pake teeds my his deep, flidy,
    There!'s we well to to who'ts here'f?
    What, man shall cound bentrose it not.



Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [23]:
import string

In [24]:
def contains_vowels(sequence):
  vowels = ["a", "e", "i", "o", "u"]
  return any([vowel in list(sequence) for vowel in vowels])

In [25]:
contains_vowels("gradient")

True

In [26]:
import numpy as np
import pandas as pd

In [27]:
sequences = []
labels = []
for i in range(1000):
 char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
 seq = "".join(char_list)
 sequences.append(seq)
 labels.append(int(contains_vowels(seq)))

In [28]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [29]:
df.head()

,sequence,label
0,lwfcd,0
1,xtqib,1
2,rkszs,0
3,mwgjh,0
4,jxgrd,0


Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [30]:
all_char = ''

for idx, row in df.iterrows():
  all_char += df.loc[idx, "sequence"]

chars = (sorted(list(set(all_char))))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print('total chars:', len(chars))

total chars: 26


In [31]:
maxlen=5

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.uint8)

for i, seq in enumerate(sequences):
  for t,char in enumerate(seq):
        X[i, t, char_indices[char]] = 1

Y = np.array(labels)

In [32]:
model = Sequential()

model.add(tfkl.LSTM(64, input_shape=(maxlen, len(chars))))
model.add(tfkl.Dense(1,activation="sigmoid"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                23296     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 23,361
Trainable params: 23,361
Non-trainable params: 0
_________________________________________________________________


In [33]:

model.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01),metrics=['accuracy'])
results = model.fit(X, Y, epochs=20, batch_size=200)

Epoch 1/20
5/5 [==============================] - 0s 3ms/step - loss: 0.6206 - accuracy: 0.6530
Epoch 2/20
5/5 [==============================] - 0s 3ms/step - loss: 0.4271 - accuracy: 0.7660
Epoch 3/20
5/5 [==============================] - 0s 3ms/step - loss: 0.2778 - accuracy: 0.8750
Epoch 4/20
5/5 [==============================] - 0s 3ms/step - loss: 0.1842 - accuracy: 0.9510
Epoch 5/20
5/5 [==============================] - 0s 3ms/step - loss: 0.1435 - accuracy: 0.9680
Epoch 6/20
5/5 [==============================] - 0s 3ms/step - loss: 0.1054 - accuracy: 0.9810
Epoch 7/20
5/5 [==============================] - 0s 3ms/step - loss: 0.0462 - accuracy: 0.9940
Epoch 8/20
5/5 [==============================] - 0s 3ms/step - loss: 0.0589 - accuracy: 0.9870
Epoch 9/20
5/5 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 10/20
5/5 [==============================] - 0s 3ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 11/20
5/5 [======================